In [1]:
code_dir = string(homedir(),"/DataScience/sne_nu/code/");
data_dir = string(homedir(),"/DataScience/sne_nu/data/");
notebook_dir = string(homedir(),"/DataScience/sne_nu/notebook/");

In [2]:
include(string(code_dir,"sig_and_bkg_pdfs.jl"));

Next we will put together functions to construct the test statistic TS. For the $j$th SN, we at attempting to determine the number of signal neutrinos ($s_j$) associated with it, compared with the expected number of associated background neutrinos ($b_j$). We accomplish this by maximizing the TS (really minimizing its negative) after computing the expected number of background using randomized neutrino samples $\langle b_j\rangle$ and the Signal and Background PDFs for the $i$th neutrino with the $j$th SN.

$$
{\rm TS} = \sum_j^{N_{\rm sn}} \left[-s_j + \sum_{i=1}^{N_j} \ln\left[\frac{s_j \mathcal{S}_i}{b_j \mathcal{B}_i} + 1\right]\right]
$$

Note that $\mathcal{S}_i$ and $\mathcal{B}_i$ are composite functions of the neutrino arrival time and direction relative to SN $j$.

$$
\mathcal{S}_i = \mathcal{S}_{T}\,\mathcal{S}_{\rm dir}
$$

This is described in more detail in the notebook "Generating Signal and Background PDFs". Because we only care about $s_j$, I combine the rest of the parameters $c_{i,j} = \frac{\mathcal{S}_i}{\langle b\rangle_j\mathcal{B}_i}$. Each sn object contains an empty list which can contain the coefficients. 




In [3]:
using StatsBase
using Distributions

In [4]:
nu_data = readdlm(string(data_dir,"cleaned_nu_data.csv"),',',header=true)[1];
sne_data = readdlm(string(data_dir,"cleaned_sne_data.csv"),',',header=true)[1];

In [5]:
# Function that reads in an array of neutrino data and converts it into an array of nu objects 
# 
# Inputs : 
#    nu_data --  a 2D array of floats containing raw neutrino data
#                data organization : [Arrival Day (MJD), Angular Error, RA, Dec]
#
# Output : 
#    an array of nu objects created from raw neutrino data 
#

function create_nu_array(nu_data::Array{Float64,2})
    return [nu(nu_data[i,:]...) for i in 1:length(nu_data[:,1])]
end

# Function that reads in raw neutrino data an returns a randomized sample 
#
# Inputs : 
#   orig_data --  a 2D array of floats containing original neutrino data 
#                 data organization : [Arrival Day (MJD), Angular Error, RA, Dec]
#
#   N_nus     --  Number of randomized samples. If no value is given use the entire length of the orig_data
#
# Output : 
#    a 2D array of floats containing a randomized sample of neutrino data 
#    data organization : [Arrival Day (MJD), Angular Error, RA, Dec]

function create_sample_nus(orig_data::Array{Float64,2},N_nus=0)
    
    if N_nus == 0 
        N_nus = length(orig_data[:,1])
    end
    
    mjd = sample(orig_data[:,1],replace=true,N_nus);
    ang_err = sample(orig_data[:,2],replace=true,N_nus);
    ra = rand(Uniform(0,2pi),N_nus);
    dec = sample(orig_data[:,4],replace=true,N_nus);

    return hcat(mjd,ang_err,ra,dec)
end


create_sample_nus (generic function with 2 methods)

In [6]:
# Function that reads in an array of SN data and converts it into an array of sn objects 
# 
# Inputs : 
#    sn_data --  a 2D array of floats containing raw neutrino data
#                data organization : [Max Date (MJD), RA, Dec, Redshift (z)]
#
# Output : 
#    an array of sn objects created from raw SN data 
#

function create_sn_array(sn_data::Array{Any,2})
    return [sn(sn_data[i,2:5]...) for i in 1:length(sn_data[:,1])];
end

create_sn_array (generic function with 1 method)

In [7]:
sne = create_sn_array(sne_data);

In [8]:
# function that calculates the average number of neutrinos associated with each SN using scrambled neutrino data 
# the results are written to a file "nb_data.dat" 
function calc_nb(N_samples = 1000)

    len_sne = length(sne); 
    len_nu = length(nu); 

    # Array to store the number of background neutrinos for each SN for each randomized sample 

    num_nb = zeros(Int,N_samples,len_sne); 

    for n in 1:N_samples
        sample_nus = create_nu_array(create_sample_nus(nu_data,len_nu))
        num_nb[n,:] = [sum(find_associated_nus(sn,sample_nus)) for sn in sne]
    end
    
    writedlm(string(data_dir,"nb_data.dat"),[mean(num_nb[:,i]) for (i,_) in enumerate(sne)])
end



calc_nb (generic function with 2 methods)

In [9]:
nbs = readdlm(string(data_dir,"nb_data.dat"))[:]; 

In [10]:
[add_nb!(sn,nbs[i]) for (i,sn) in enumerate(sne)];

In [11]:
nus = create_nu_array(nu_data);

In [12]:
[add_nu!(sn,nus[find_associated_nus(sn,nus)]) for (i,sn) in enumerate(sne)];

In [13]:
function calc_coefs!(t_sn::sn)
    if length(t_sn.associated_nus) > 0
        
        temp_s = map(x-> S_dir(t_sn,x),t_sn.associated_nus);
        temp_s .*= map(x-> S_time(t_sn,x),t_sn.associated_nus);

        temp_b = (1/(2*pi))*map(B_dec,[t_sn.associated_nus[j].dec for j in 1:length(t_sn.associated_nus)]);
        temp_b ./= 16.0;
        
        add_coefs!(t_sn,temp_s./temp_b./t_sn.nb);
    end
end

calc_coefs! (generic function with 1 method)

In [15]:
map(calc_coefs!,sne);

In [17]:
sne[1].coefs

9-element Array{Float64,1}:
  0.0812887
 23.9836   
  0.912558 
  1.48972  
  5.4768   
  0.0982267
  0.115154 
  0.381794 
  0.24065  